In [1]:
import requests
import urlparse
import datetime
from datetime import date

tsvBaseURL='https://proba-v-mep.esa.int/api/timeseries/v1.0/ts/'
owsBaseURL='http://mapcachedev.vgt.vito.be/geoserver/PV_MEP/ows'

def pointTSRequest(coverageId, lon, lat, startDate = None, endDate = None):
    
    tsvURL = urlparse.urljoin(tsvBaseURL, coverageId + '/')
    tsvURL = urlparse.urljoin(tsvURL, 'point')
    
    payload = {
        'lon': str(lon),
        'lat': str(lat)
    }    
    
    if startDate != None:
        payload['startDate'] = startDate.strftime('%Y-%m-%d')
    if endDate != None:
        payload['endDate'] = endDate.strftime('%Y-%m-%d')
        
    return requests.get(tsvURL, params=payload)

def pointTS(coverageId, lon, lat, startDate = None, endDate = None):
    
    response = pointTSRequest(coverageId, 10, 10, startDate, endDate)
    timeSeries = response.json()['results']    
    timeSeries = map(lambda x:[datetime.datetime.strptime(x['date'], "%Y-%m-%d").date(),x['result']['average']], 
                     timeSeries)
    return timeSeries    
    

In [2]:
ts = pointTS('PROBAV_L3_S10_TOC_NDVI_333M', 10, 10)

In [3]:
import sys
sys.path
! pip install --user mpld3
if '/home/daemsd/.local/lib/python2.6/site-packages/' not in sys.path:
    sys.path.append('/home/daemsd/.local/lib/python2.6/site-packages/')
sys.path

DEPRECATION: Python 2.6 is no longer supported by the Python core team, please upgrade your Python. A future version of pip will drop support for Python 2.6


['',
 '/usr/hdp/current/spark-client/python',
 '/usr/hdp/current/spark-client/python/lib/py4j-0.9-src.zip',
 '/usr/local/lib/python27.zip',
 '/usr/local/lib/python2.7',
 '/usr/local/lib/python2.7/plat-linux2',
 '/usr/local/lib/python2.7/lib-tk',
 '/usr/local/lib/python2.7/lib-old',
 '/usr/local/lib/python2.7/lib-dynload',
 '/usr/local/lib/python2.7/site-packages',
 '/usr/local/lib/python2.7/site-packages/setuptools-28.8.0-py2.7.egg',
 '/usr/local/lib/python2.7/site-packages/pip-9.0.1-py2.7.egg',
 '/usr/local/lib/python2.7/site-packages/GDAL-1.11.2-py2.7-linux-x86_64.egg',
 '/usr/local/lib/python2.7/site-packages/IPython/extensions',
 '/home/daemsd/.ipython',
 '/home/daemsd/.local/lib/python2.6/site-packages/']

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
fig.set_size_inches(12, 4, forward=True)
timeStamps = map(lambda x:x[0], ts)
values = map(lambda x:x[1], ts)

scatter = ax.scatter(timeStamps, values)

import mpld3

labels = ['{0} - {1}'.format(i[1],i[0]) for i in ts]
tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
mpld3.plugins.connect(fig, tooltip)

mpld3.display(fig)